In [12]:
# typing — Support for gradual typing as defined by PEP 484 and subsequent PEPs.
from typing import TypeAlias, Literal, List, Tuple, Callable, Optional

# NumPy — Funções para computação numérica.
import numpy as np

# Numba — Aceleração do Python via JIT.
from numba import njit

# matplotlib — An object-oriented plotting library.
import matplotlib.pyplot as plt

# copy — Deep and shallow copies.
from copy import deepcopy as copy

In [13]:
@njit
def rastrigin(x):
	n = len(x)
	A = 10
	return A * n + np.sum(x ** 2 - A * np.cos(2 * np.pi * x))

rastrigin(np.array([0, 0, 0, 0, 0, 0, 0]))

0.0

In [14]:
def create_individual(size: int, domain: Tuple[float, float]) -> np.ndarray:
	"""
	Cria um indivíduo aleatório.
	"""

	return np.random.uniform(*domain, size = size)

create_individual(20, (-10, 10))

array([-4.97278461, -2.18682141, -6.34870853,  4.71741328, -6.25573629,
        3.05168195,  5.04823689, -6.43336142,  7.59564056,  4.44379914,
       -3.48796257, -2.80221524,  7.12785684,  2.94947157, -3.04542723,
       -6.29234457,  2.75967412, -8.93917248, -3.85518409,  4.24622201])

In [15]:
@njit
def tournament_selection(population: np.ndarray, fitnesses: np.ndarray, k: int) -> np.ndarray:
	"""
	Realiza a seleção por torneio.
	"""

	# Selecionar aleatoriamente 'k' indivíduos para o torneio.
	tournament_indices = np.random.choice(population.shape[0], k, replace = False)
	
	# Inicie com o primeiro (poderia ser qualquer um).
	best_index = tournament_indices[0]

	# Encontre o índice do indivíduo com a melhor aptidão entre os selecionados.
	for idx in tournament_indices[1:]:
		if fitnesses[idx] > fitnesses[best_index]:
			best_index = idx
	
	# Retorne o indivíduo com a melhor aptidão.
	return population[best_index]

In [16]:
@njit
def simulated_binary_crossover(a: np.ndarray, b: np.ndarray, eta: float, domain: Tuple[float, float]) -> Tuple[np.ndarray, np.ndarray]:
	
	# Inicialize a próle.
	offspring1 = np.empty_like(a)
	offspring2 = np.empty_like(b)

	# Estes são os limites inferior e superior.
	lower_bound, upper_bound = domain
	
	# Faça o crossover para cada elemento.
	for i in range(a.shape[0]):

		# ~U(0, 1)
		r = np.random.rand()
		
		# Caso `r` seja <= 0.5..
		if r <= 0.5:
			gamma = (2 * r) ** (1 / (eta + 1))
		else:
			gamma = (1 / (2 * (1 - r))) ** (1 / (eta + 1))
		
		offspring1[i] = 0.5 * ((1 + gamma) * a[i] + (1 - gamma) * b[i])
		offspring2[i] = 0.5 * ((1 - gamma) * a[i] + (1 + gamma) * b[i])
		
	# Garanta que a próle está dentro do limite.
	offspring1 = np.clip(offspring1, a_min = lower_bound, a_max = upper_bound)
	offspring2 = np.clip(offspring2, a_min = lower_bound, a_max = upper_bound)

	return offspring1, offspring2


In [17]:
simulated_binary_crossover(np.array([0.0, 0.0]), np.array([1.0, 1.0]), eta = 1, domain = (-1, 1))

(array([0.11135676, 0.21290151]), array([0.88864324, 0.78709849]))

In [18]:
@njit
def mutate(individual: np.ndarray, rate: float, eta: float, domain: Tuple[float, float]) -> np.ndarray:
	
	n = np.random.randn(individual.shape[0])

	lower_bound, upper_bound = domain
	
	for i in range(individual.shape[0]):
		if np.random.uniform(0, 1) <= rate:
			
			# Mutação Delta: https://arxiv.org/pdf/1508.00468
			# delta = (2 * np.random.rand() - 1) * (1 - abs(individual[i] - lower_bound) / (upper_bound - lower_bound)) ** (1 / (eta + 1))
			# individual[i] += delta

			# Mutação Gaussiana.
			individual[i] += eta * n[i]
	
	# Garantir que os valores estejam dentro dos limites do domínio.
	individual = np.clip(individual, lower_bound, upper_bound)
	
	return individual

In [19]:
mutate(np.array([0., -1., 4.]), rate = 0.99, eta = 0.1, domain = (-10, 10))

array([-0.03729055, -0.91059174,  3.9507934 ])

In [20]:
@njit
def has_converged(population: np.ndarray, threshold: float = 1e-2) -> np.bool:
	"""
	A população convergiu caso todos sejam o mesmo indivíduo.
	"""

	return np.var(population) < threshold

has_converged(np.array([
	np.array([2]),
	np.array([2.1])
]))

True

In [23]:
def non_canonic_genetic_algorithm(f: Callable, P: int, p: int, domain: Tuple[float, float], generations: int = 1_000, rounds: int = 100, mutation_rate: float = 0.01, n_children: int = 2, eta: float = 1, k: int = 2):
	
	# Lista de todas as aptidões.
	all_fitnesses = []

	for _ in range(rounds):

		# População inicial aleatória.
		population = np.array([create_individual(p, domain) for _ in range(P)])

		for _ in range(generations):

			# Esta é uma lista de todas as aptidões.
			fitnesses = np.array([f(individual) for individual in population])

			# Esta é a nova população.
			new_population = []
			
			while len(new_population) < P:
					
				# Selecione dois indivíduos com o método do torneio.
				parent1 = tournament_selection(population, fitnesses, k = k)
				parent2 = tournament_selection(population, fitnesses, k = k)
				
				# Estes são sua próle.
				offspring1, offspring2 = simulated_binary_crossover(parent1, parent2, eta = eta, domain = domain)
				
				# Realize a mutação da prole.
				offspring1 = mutate(offspring1, mutation_rate, eta, domain)
				offspring2 = mutate(offspring2, mutation_rate, eta, domain)

				# Adicione as próles à nova população.
				new_population.append(offspring1)
				new_population.append(offspring2)
			
			# A nova população substitui a "antiga".
			population = np.array(new_population)

			# Adicione a melhor aptidão às melhores aptidões.
			all_fitnesses += fitnesses.tolist()

		# Caso a população tenha convergido, pare.
		if has_converged(population):
			break
	
	return all_fitnesses

In [39]:
fitnesses = non_canonic_genetic_algorithm(
	
	# Definições do Problema.
	f = lambda x: rastrigin(x),
	p = 20,
	domain = (-10, 10),
	
	# Total de Rodadas
	rounds = 100,

	# Hiperparâmetros Gerais.
	generations = 100,
	P = 50,

	# Hiperparâmetros Específicos.
	eta = 1.0,
	mutation_rate = 0.02,
    k = 10
	
)

print("\nMelhor Aptidão:", np.min(fitnesses))
print("Pior Aptidão:", np.max(fitnesses))
print("Aptidão Média:", np.mean(fitnesses))
print("np.std(Aptidão):", np.std(fitnesses))


Melhor Aptidão: 416.6908326783073
Pior Aptidão: 2213.217171843826
Aptidão Média: 2041.1269972592138
np.std(Aptidão): 256.4267913532014
